## Environment

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Basic
import os
import re
import sys
import glob
import numpy as np
import pandas as pd

import itertools
import more_itertools
from pathlib import Path
from pprint import pprint
from collections import defaultdict, Counter
from typing import (List, Dict, Any, NoReturn, 
                    Tuple, Optional, Union)
from tqdm import tqdm_notebook

import multiprocessing
from multiprocessing_logging import install_mp_handler

import logging
logging.basicConfig(level=logging.DEBUG,
                   format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

import warnings
warnings.filterwarnings('ignore')

print(f"Number CPU: {multiprocessing.cpu_count()}")

Number CPU: 12


In [3]:
# W2V
import spacy
import pymorphy2
from gensim.utils import any2utf8, to_utf8
from gensim.models import KeyedVectors
from gensim.models.phrases import Phrases
from nltk.stem.snowball import SnowballStemmer
from gensim.models.phrases import Phrases, npmi_scorer
from gensim.models import word2vec, keyedvectors

### Define paths

In [4]:
BASE_DIR = Path("..")
DATA_DIR  = BASE_DIR / "data"
LISTS_DIR  = BASE_DIR / "lists"
MODEL_DIR  = BASE_DIR / "models"

## Load data

In [5]:
df = []
for fn in tqdm_notebook(glob.glob(str(DATA_DIR / "*.csv"))):
    df_part = pd.read_csv(fn, sep=';', encoding='utf-8')
    print(f"Data part of shape: {df_part.shape}")
    df.append(df_part)
df = pd.concat(df).reset_index(drop=True)
print(f"All data of shape: {df.shape}")
df.head()

Data part of shape: (3944, 10)
Data part of shape: (9200, 10)

All data of shape: (13144, 10)


,incident,req_reason,req_reg_datetime,req_num,msg,product,subproduct,subject,s_subject,day_of_the_year
0,OTHER,Инцидент,2020-11-05 11:27:21,2011050726933001,здравствуйте здравствуйте я застрахованное лиц...,Страхование,.Страховой случай,Консультация по продуктам и обслуживанию,"Разъяснения (условия, сроки, статусы рассмотре...",[310]
1,OTHER,Инцидент,2020-11-19 15:27:04,2011190004665301,Куда #ТОПОНИМ мой аватар Ну если не понял то...,Онлайн-сервисы,Функционирование МБ/СБОЛ/МП,Работа в системе,Информация по возможностям/ограничениям,[324]
2,OTHER,Инцидент,2020-11-20 13:16:35,2011200083620001,меня зовут #ФИО здравствуйте здравствуйте у ...,Физ. лица - иные услуги/продукты,Кредитная история (БКИ),Вопросы по отчету,Отчет не поступил/поступил с ошибкой,[325]
3,OTHER,Инцидент,2020-11-21 19:29:17,2011210190303001,#ФИО здравствуйте добрый день я вас слушаю я...,Онлайн-сервисы,Функционирование МБ/СБОЛ/МП,Работа в системе,Информация по возможностям/ограничениям,[326]
4,IM0104549077,Инцидент,2020-11-20 13:40:55,2011200086706501,меня зовут #ФИО здравствуйте #ФИО добрый д...,Онлайн-сервисы,Функционирование МБ/СБОЛ/МП,Работа в системе,Некорректная работа Мобильного Приложения СБОЛ,[325]


## Configure preprocessing

In [6]:
def file_opener(filename: str) -> str:
    with open(filename, 'rt', encoding='utf-8-sig') as src:
        file = src.read()
    return "|".join([x for x in file.split('\n') if x])


class DataPreprocessorLemmatizer:

    text_features = ['msg']
    stopgrams = [
             'CONJ',   # союз
             'PRCL',   # частица
             'PRED',   # предикатив
             'NPRO',   # местоимение-сущ.
             'INTJ',   # междометие
             'Erro',   # ошибка
             'Dist',   # искажение
             'Ques',   # вопросительное слово
             'Dmns',   # указательное слово
             'Prnt'   # вводное слово
            ]

    def __init__(self, multipocess: bool, num_processors: int=16, chunksize: int=100, 
                intro_words_path: str = './lists'):
        # Language parsers
        self.__morph = pymorphy2.MorphAnalyzer()
        self.__nlp = spacy.blank('ru')
        # Multiprocessing params
        self.__multipocess = multipocess
        self.__num_processors = num_processors
        self.__chunksize = chunksize
        # Cleaning utils
        self.__intro_words = file_opener(os.path.join(intro_words_path, 'intro.txt'))
        self.__nltk_stopwords = file_opener(os.path.join(intro_words_path, 'NLTK_stopwords.txt')).split("|")

    def get_stopwords(self):
        """ 
        Check intro-words list.
        """
        return self.__nltk_stopwords

    def get_intro_words(self):
        """ 
        Check intro-words list.
        """
        return self.__intro_words
    
    def get_analyzer(self):
        """
        Allow to access to Pymorphy Analyzer instance.
        """
        return self.__morph

    
    def _process_text(self, text: str):
        """ 
        Process single text and return list of tokens.
        """
        if pd.isna(text):
            return []
       # Pre-processing part 
        text = [str(token).lower()
                for token in self.__nlp.make_doc(text)
                if (token and token.is_alpha and len(str(token.text)) > 2 and ~token.is_stop)]
        # Processing part
        clean_text = []
        for token in text:
            token = self.__morph.parse(str(token).lower())[0]
            if ((token.normal_form not in self.__nltk_stopwords) 
                and (token.normal_form not in self.__intro_words)
                and all([tag not in token.tag for tag in self.stopgrams])):
                clean_text.append(token.normal_form)
        return clean_text
    

    def process_texts(self, texts: List[str]):
        """ 
        Process list of texts and return list of lists of tokens.
        """
        if self.__multipocess:
            with multiprocessing.Pool(self.__num_processors) as pool:
                processed_texts = list(tqdm_notebook(pool.imap(self._process_text, texts, 
                                                               chunksize=self.__chunksize), 
                                                     total=len(texts)))
            return processed_texts
        else:
            return [self._process_text(text) for text in tqdm_notebook(texts)]
        


    def process(self, data: pd.DataFrame,
                features_cols: Optional[List[str]] = None, copy: bool=True) -> pd.DataFrame:
        """
        Preprocess text for language modelling.
         - clean introduction words, numbers and small prefixes;
         - tokenize and lemmatize texts;
        """
        logging.info("Text processing started.")
        if not features_cols:
            features_cols = self.text_features

        for col_name in features_cols:
            logging.info(f"Processing '{col_name}' column...")
            data_processed = self.process_texts(data[col_name].fillna("").to_list())
            if not copy:
                data[col_name] = data_processed
            else:
                data[col_name + "_proc"] = data_processed

        logging.info("Text preprocessing finished.")
        return data

In [7]:
processor = DataPreprocessorLemmatizer(multipocess=False,
                                       num_processors=8, chunksize=200,
                                       intro_words_path=str(LISTS_DIR))

2021-07-21 09:13:35,098 - pymorphy2.opencorpora_dict.wrapper - INFO - Loading dictionaries from C:\Users\airen\Anaconda3\envs\pycharmenv\lib\site-packages\pymorphy2_dicts\data
2021-07-21 09:13:35,172 - pymorphy2.opencorpora_dict.wrapper - INFO - format: 2.4, revision: 393442, updated: 2015-01-17T16:03:56.586168
2021-07-21 09:13:35,185 - pymorphy2.opencorpora_dict.wrapper - INFO - Loading dictionaries from C:\Users\airen\Anaconda3\envs\pycharmenv\lib\site-packages\pymorphy2_dicts\data
2021-07-21 09:13:35,233 - pymorphy2.opencorpora_dict.wrapper - INFO - format: 2.4, revision: 393442, updated: 2015-01-17T16:03:56.586168


In [8]:
%%time
df = processor.process(data=df, features_cols=['msg'])

2021-07-21 09:13:35,481 - root - INFO - Text processing started.
2021-07-21 09:13:35,481 - root - INFO - Processing 'msg' column...


2021-07-21 09:20:02,720 - root - INFO - Text preprocessing finished.



Wall time: 6min 27s


In [9]:
for i, row in df[['msg', 'msg_proc']].head().iterrows():
    print("--------------")
    print(f"{row['msg']}\n{row['msg_proc']}")

--------------
здравствуйте здравствуйте я застрахованное лицо я хотела бы узнать договор полиса и дата оплаты страховой премии мне по поводу кража у меня украли  #ЧИСЛО  тысяч мошенники я как бы чтоб страховой написать заявление то есть подать на страховой случай я правильно понимаю да да да мне должны указать договор полис я по телефону это все как то оплачивала договор вам оплатить нужно правильно понимаю нет у меня мошенники сняли  #ЧИСЛО  тысяч и как бы я хочу в страховую службу написать заявление мне нужно указать договор полис и дата оплаты страховой премии когда оплачивалась то есть вам нужен номер договора да номер договора мы не сообщаем как же мне заявление для по поводу кражи вы сказали вам звонить в сбербанк щас сижу так одна минута деньги  #ЧИСЛО  когда позвонит еще послан от обратиться этот номер договора номер договора говорит не сообщаем а потом и одну минуточку подождите не сообщаем а потом что мне делать и для документации не ладно сейчас минуточку подождите сейчас в

In [10]:
class Bigrammer:
    
    def __init__(self, phrase_model=None):
        self.__phrase_model = phrase_model
    
    def train(self, texts: List[List[str]], 
              min_count: int, threshold: float,
             to_save: bool, save_path: str=".", phrases_fn: str="phrases.pkl"):
        """
        Train gensim Phrases model with NPMI scorer.
        :param texts - The training corpus must be a sequence of sentences,
                        with each sentence a list of tokens.
        :param min_count – Ignore all words and bigrams with total 
                            collected count lower than this value.
        :param threshold – Represent a score threshold for forming 
                            the phrases (higher means fewer phrases). 
                            A phrase of words a followed by b is accepted if the score of 
                            the phrase is greater than threshold. 
                            For NPMI scorer is in the range -1 to 1.
        """
        logging.info("Training bigrammer started.")
        self.__phrase_model = Phrases(texts, min_count=min_count, 
                               threshold=threshold, scoring='npmi')
        logging.info("Training bigrammer finished.")
        if to_save:
            self.__phrase_model.save(os.path.join(save_path, phrases_fn))
            if os.path.isfile(os.path.join(save_path, phrases_fn)):
                logging.info(f"Bigrammer model successfully saved to: {os.path.join(save_path, phrases_fn)}")
        return self
    
    @classmethod
    def load(cls, save_path: str, phrases_fn: str) -> object:
        """
        Load pre-trained model from file and init.
        """
        if os.path.isfile(os.path.join(save_path, phrases_fn)):
            logging.info(f"Bigrammer model loading from: {os.path.join(save_path, phrases_fn)}")
        phrase_model = Phrases.load(os.path.join(save_path, phrases_fn))
        logging.info(f"Bigrammer model successfully loaded.")
        return cls(phrase_model=phrase_model)
    
    
    def create_bigramms(self, texts: List[List[str]]) -> List[List[str]]:
        """
        Create bi-gramms from given text data, already splitted.
        """
        return [self.__phrase_model[text] if len(text) > 0 
                else [] for text in tqdm_notebook(texts)]
    
    
    def process(self, data: pd.DataFrame,
                text_col: str, copy: bool=True) -> pd.DataFrame:
        """
        Create bi-gramms from given column in dataframe.
        """
        logging.info(f"Bigramms creation for texts in column {text_col} started")
        data_processed = self.create_bigramms(data[text_col].fillna("").to_list())
        if not copy:
            data[text_col] = data_processed
        else:
            data[text_col + "_bigramms"] = data_processed

        logging.info("Bigramms creation finished.")
        return data
    
    
    def get_vocab(self) -> Dict[bytes, int]:
        logging.info(f"Bigrammer vocab size: {len(self.__phrase_model.vocab)}")
        return self.__phrase_model.vocab
    
    
    def get_phraser(self) -> Phrases:
        return self.__phrase_model

In [11]:
bigrammer = Bigrammer().train(df['msg_proc'].to_list(),
                             min_count=5, threshold=0.3,
                             to_save=False)
bi_vocab = bigrammer.get_vocab()

2021-07-21 09:20:03,185 - root - INFO - Training bigrammer started.
2021-07-21 09:20:03,186 - gensim.models.phrases - INFO - collecting all words and their counts
2021-07-21 09:20:03,186 - gensim.models.phrases - INFO - PROGRESS: at sentence #0, processed 0 words and 0 word types
2021-07-21 09:20:04,566 - gensim.models.phrases - INFO - PROGRESS: at sentence #10000, processed 873920 words and 251328 word types
2021-07-21 09:20:04,984 - gensim.models.phrases - INFO - collected 303577 word types from a corpus of 1148128 words (unigram + bigrams) and 13144 sentences
2021-07-21 09:20:04,984 - gensim.models.phrases - INFO - using 303577 counts as vocab in Phrases<0 vocab, min_count=5, threshold=0.3, max_vocab_size=40000000>
2021-07-21 09:20:04,985 - root - INFO - Training bigrammer finished.
2021-07-21 09:20:04,985 - root - INFO - Bigrammer vocab size: 303577


In [12]:
bigramms = [k for k in bi_vocab.keys() if "_" in k.decode('utf-8')]
print(f"Bigramms in vocab: {len(bigramms)} from {len(bi_vocab)} which is {100*len(bigramms)/len(bi_vocab)}\n")

for i, (k, v) in enumerate(bi_vocab.items()):
    print(f"{k.decode('utf-8')} --> {v}")
    if i > 10:
        break

Bigramms in vocab: 288359 from 303577 which is 94.98710376609559

застраховать --> 135
лицо --> 733
застраховать_лицо --> 7
хотеть --> 7220
лицо_хотеть --> 4
узнать --> 2319
хотеть_узнать --> 1013
договор --> 681
узнать_договор --> 2
полис --> 516
договор_полис --> 4
дата --> 2175


In [13]:
%%time
df = bigrammer.process(df, text_col='msg_proc', copy=True)

2021-07-21 09:20:05,368 - root - INFO - Bigramms creation for texts in column msg_proc started


2021-07-21 09:20:09,777 - root - INFO - Bigramms creation finished.



Wall time: 4.41 s


In [14]:
for i, row in df[['msg', 'msg_proc_bigramms']].head().iterrows():
    print("--------------")
    print(f"{row['msg']}\n{row['msg_proc_bigramms']}")

--------------
здравствуйте здравствуйте я застрахованное лицо я хотела бы узнать договор полиса и дата оплаты страховой премии мне по поводу кража у меня украли  #ЧИСЛО  тысяч мошенники я как бы чтоб страховой написать заявление то есть подать на страховой случай я правильно понимаю да да да мне должны указать договор полис я по телефону это все как то оплачивала договор вам оплатить нужно правильно понимаю нет у меня мошенники сняли  #ЧИСЛО  тысяч и как бы я хочу в страховую службу написать заявление мне нужно указать договор полис и дата оплаты страховой премии когда оплачивалась то есть вам нужен номер договора да номер договора мы не сообщаем как же мне заявление для по поводу кражи вы сказали вам звонить в сбербанк щас сижу так одна минута деньги  #ЧИСЛО  когда позвонит еще послан от обратиться этот номер договора номер договора говорит не сообщаем а потом и одну минуточку подождите не сообщаем а потом что мне делать и для документации не ладно сейчас минуточку подождите сейчас в

## TextRank

In [15]:
import networkx as nx

In [18]:
class Candidate(object):
    """
    The keyphrase candidate data structure.
    """

    def __init__(self, token: str, 
                 is_bigramm: bool,
                 pos: pymorphy2.tagset.OpencorporaTag,
                 score: float,
                 lexical_forms: Dict[str, str]=None,
                 bigramm_sep: str="_"):

        self._token = token
        self._is_bigramm = is_bigramm
        self._pos = pos
        self._score = score
        
        self._lexical_forms = lexical_forms 
        self._bigramm_splitted = [] # TODO: split bigramm
        
        if self._is_bigramm:
            self._bigramm_splitted = self._token.split(bigramm_sep)

In [47]:
class TextRanker:
    """
    TextRank for keyword extraction.
    This model builds a graph that represents the text. A graph based ranking
    algorithm is then applied to extract the lexical units (here the words) that
    are most important in the text.
    In this implementation, 
     - nodes - are words of certain part-of-speech (nouns/adjectives/..) 
     - edges - represent co-occurrence relation, controlled by the distance 
               between word occurrences - a window of N words). 
    """
    rus_lexical_forms = ['nomn', 'gent', 'datv', 'accs', 'ablt', 'loct', 'voct']

    def __init__(self):
        # From pymorphy2 avaliable POS tags
        # ref: http://opencorpora.org/dict.php?act=gram 
        self.__pos_list = ['NOUN', 'ADJS', 'ADJF', 'COMP', 'VERB', 'INFN', 
                           'PRTF', 'PRTS', 'GRND', 'NUMR', 'ADVB', 'Abbr']
        
        # Russian language parser
        self.__morph = pymorphy2.MorphAnalyzer()
        # Words graph
        self.__graph = nx.Graph()
        self.__texts = []  # as List[List[Dict[str, Any]]] -> [[{'words': [], 'bigramm': [], 'POS': []}]]
        # Each inner Dict == single token
        # Each inner List == single text
        # Outer List is composition of texts
        
        # Keyphrase candidates container
        self.__candidates = defaultdict(Candidate)
        

        
    def candidate_weighting(self, texts: List[List[str]],
                            window: int=2, pos_list: List[str]=None,
                            include_bigramms: bool=True,
                            top_percent: float=None):
        """
        Tailored candidate ranking method for TextRank. 
        Keyphrase candidates are either composed from the T-percent (top_percent) 
        highest-ranked words or extracted using the `candidate_selection()` method.
        Candidates are ranked using the sum of their words.
        :param window - the window for connecting words in the graph.
        :param pos_list - the set of valid pos for words to be considered as nodes
                    in the graph, defaults to ('NOUN', 'PROPN', 'ADJ').
        :param top_percent - percentage of top vertices to keep for phrase generation.
        """
        if pos_list is not None:
            self.__pos_list = pos_list
            
        # flatten document as a sequence of (word, bigramm, pos) samples
        self.__texts = self.__tag_words(texts)
        self.__window = window
        self.__include_bigramms = include_bigramms
        self.__build_word_graph()
        
        # Computes the word scores using the unweighted PageRank formula
        # pagerank_scipy() is a SciPy sparse-matrix implementation of the power-method
        # Returns: pagerank – Dictionary of nodes with PageRank as value
        textranked = nx.pagerank_scipy(self.__graph, alpha=0.85, tol=0.0001, weight=None)
        
        # Generate the phrases from the T-percent top ranked words
        if top_percent is not None:

            # warn user as this is not the pke way of doing it
            logging.info(f"Candidates are generated using {top_percent}%-top")

            # computing the number of top keywords
            n_nodes = self.__graph.number_of_nodes()
            to_keep = min(int(n_nodes * top_percent), n_nodes)

            # Sorting the nodes by decreasing scores
            top_words = {k: v for k, v in sorted(textranked.items(), key=lambda item: item[1], reverse=True)}
            
            # Create candidates
            self.__create_candidates(top_words)
    
    
    def __check_validness(self, token_dict: Dict[str, Any]) -> bool:
        """
        Mark token `valid` if it belongs to one of selected POS or it is a bigramm.
        """
        return token_dict['bigramm'] or (token_dict['pos'] is not None and any([tag in token_dict['pos'] 
                                         for tag in self.__pos_list]))
        
        
    def __tag_words(self, texts: List[List[str]]):
        """
        Process given texts to selected form: 
        [[{'words': [], 'bigramm': [], 'POS': [], 'valid'}]]
        """
        texts = [[{'token': token,
                  'bigramm': True if "_" in token else False,
                  'pos': self.__morph.parse(str(token).lower())[0].tag if "_" in token else None}
                 for token in tokens] 
                for tokens in tqdm_notebook(texts)]
        _ = [[token.update({'valid': self.__check_validness(token)})  
                 for token in tokens] for tokens in tqdm_notebook(texts)]
        return texts
            
            
    def __build_word_graph(self):
        """
        Build a graph representation of the document in which nodes/vertices
        are words and edges represent co-occurrence relation. Syntactic filters
        can be applied to select only words of certain Part-of-Speech.
        Co-occurrence relations can be controlled using the distance between
        word occurrences in the document.
        """
        tokens = itertools.chain.from_iterable(self.__texts)
        # add nodes to the graph
        logging.info(f"Adding nodes to graph...")
        self.__graph.add_nodes_from([token['token'] for token in tokens if token['valid']])

        # add edges to the graph
        logging.info(f"Adding edges...")
        for text_i, tokens in enumerate(self.__texts):
            for token_i, token in enumerate(tokens):
                # speed up things
                if not token['valid']:
                    continue
                start_ind = min(token_i, (self.__window - 1) // 2)
                end_ind = min(i + self.__window, len(tokens))
                for j in range(start_ind, end_ind):
                    linked_token = tokens[j]
                    if linked_token['valid'] and linked_token['token'] != token['token']:
                        self.__graph.add_edge(token['token'], linked_token['token'])
        logging.info(f"Adding edges ended.")
        
        
    def __create_candidates(self, scored_tokens: Dict[str, float]):
        """
        Create keywords dict for all texts.
        """        
        for token, score in tqdm_notebook(scored_tokens.items()):
            self.__add_candidate(token, score)
        logging.info(f"Keywords candidates created: {len(self.__candidates)}")
        
        
    def __add_candidate(self, token: str, score: float=None):
        """
        Add a keyphrase candidate to the candidates container.
        """
        is_bigramm = True if "_" in token else False
        lexical_forms = defaultdict(str)
        parsed_token = self.__morph.parse(str(token).lower())[0]
        pos = parsed_token.tag
        if ~is_bigramm and (('NOUN' in pos) or ('ADJF' in pos) 
                            or ('NUMR' in pos) or ('PRTF' in pos)):
            for form in self.rus_lexical_forms:
                lexical_forms[form] = parsed_token.inflect({form}).word
        pos = parsed_token.tag
        self.__candidates[token] = Candidate(token, is_bigramm, pos, lexical_forms, score)
        
        
    def get_candidates(self) -> Dict[str, Any]:
        return self.__candidates
    
    def select_keywords_from_text(self, text: List[str], return_scores: bool=False) -> List[str]:
        return [candidate for candidate in self.__candidates if candidate in text]
            
    def select_keywords_from_texts(self, texts: List[List[str]], return_scores: bool=False) -> List[List[str]]:
        return [self.select_keywords_from_text(text) for text in texts]

In [43]:
textrank = TextRanker()
textrank.candidate_weighting(texts=df['msg_proc_bigramms'].to_list(),
                            window=5, include_bigramms=True,
                            top_percent=0.2)

2021-07-21 13:16:26,941 - pymorphy2.opencorpora_dict.wrapper - INFO - Loading dictionaries from C:\Users\airen\Anaconda3\envs\pycharmenv\lib\site-packages\pymorphy2_dicts\data
2021-07-21 13:16:27,011 - pymorphy2.opencorpora_dict.wrapper - INFO - format: 2.4, revision: 393442, updated: 2015-01-17T16:03:56.586168


2021-07-21 13:17:36,609 - root - INFO - Adding nodes to graph...
2021-07-21 13:17:36,759 - root - INFO - Adding edges...


2021-07-21 13:17:37,516 - root - INFO - Adding edges ended.
2021-07-21 13:17:37,766 - root - INFO - Candidates are generated using 0.2%-top


2021-07-21 13:17:40,208 - root - INFO - Keywords candidates created: 3669


In [44]:
keywords_candidates = textrank.get_candidates()
keywords_candidates

defaultdict(__main__.Candidate,
            {'сбербанк_онлайн': <__main__.Candidate at 0x2bd9ae6d7b8>,
             'личный_кабинет': <__main__.Candidate at 0x2bd9ae6d710>,
             'номер_телефон': <__main__.Candidate at 0x2bd9ae6d630>,
             'мобильный_приложение': <__main__.Candidate at 0x2bd9ae6d908>,
             'хотеть_узнать': <__main__.Candidate at 0x2bd9ae6dc88>,
             'кредитный_карта': <__main__.Candidate at 0x2bd9ae6d748>,
             'приносить_извинение': <__main__.Candidate at 0x2bd9ae6d6d8>,
             'оставаться_линия': <__main__.Candidate at 0x2bd9ae6d5f8>,
             'приложение_сбербанк': <__main__.Candidate at 0x2bd9ae6d978>,
             'последний_цифра': <__main__.Candidate at 0x2bd9ae6d780>,
             'данный_момент': <__main__.Candidate at 0x2bd9ae6d7f0>,
             'технический_причина': <__main__.Candidate at 0x2bd9ae6d588>,
             'номер_карта': <__main__.Candidate at 0x2bd9ae6d550>,
             'перевести_деньга': <__ma

In [46]:
df['keywords'] = df.msg_proc_bigramms.apply(textrank.select_keywords_from_text)
df.head()

,incident,req_reason,req_reg_datetime,req_num,msg,product,subproduct,subject,s_subject,day_of_the_year,msg_proc,msg_proc_bigramms,keywords
0,OTHER,Инцидент,2020-11-05 11:27:21,2011050726933001,здравствуйте здравствуйте я застрахованное лиц...,Страхование,.Страховой случай,Консультация по продуктам и обслуживанию,"Разъяснения (условия, сроки, статусы рассмотре...",[310],"[застраховать, лицо, хотеть, узнать, договор, ...","[застраховать_лицо, хотеть_узнать, договор, по...","[хотеть_узнать, приносить_извинение, техническ..."
1,OTHER,Инцидент,2020-11-19 15:27:04,2011190004665301,Куда #ТОПОНИМ мой аватар Ну если не понял то...,Онлайн-сервисы,Функционирование МБ/СБОЛ/МП,Работа в системе,Информация по возможностям/ограничениям,[324],"[аватар, понять, позвать, оператор, шаблон, оп...","[аватар, понять, позвать_оператор, шаблон, опе...","[скоро_вернуться, временной_ограничение, позва..."
2,OTHER,Инцидент,2020-11-20 13:16:35,2011200083620001,меня зовут #ФИО здравствуйте здравствуйте у ...,Физ. лица - иные услуги/продукты,Кредитная история (БКИ),Вопросы по отчету,Отчет не поступил/поступил с ошибкой,[325],"[звать, вопрос, характер, сбербанк, онлайн, от...","[звать, вопрос, характер, сбербанк_онлайн, отк...","[сбербанк_онлайн, идти_технологический, кредит..."
3,OTHER,Инцидент,2020-11-21 19:29:17,2011210190303001,#ФИО здравствуйте добрый день я вас слушаю я...,Онлайн-сервисы,Функционирование МБ/СБОЛ/МП,Работа в системе,Информация по возможностям/ограничениям,[326],"[слушать, заходить, свой, сбербанк, приложение...","[слушать, заходить, свой, сбербанк, приложение...","[московский_время, код_ошибка, вестись_техниче..."
4,IM0104549077,Инцидент,2020-11-20 13:40:55,2011200086706501,меня зовут #ФИО здравствуйте #ФИО добрый д...,Онлайн-сервисы,Функционирование МБ/СБОЛ/МП,Работа в системе,Некорректная работа Мобильного Приложения СБОЛ,[325],"[звать, хотеть, спросить, сегодня, делать, пер...","[звать, хотеть_спросить, сегодня, делать_перев...","[сбербанк_онлайн, личный_кабинет, данный_момен..."


In [48]:
for i, row in df[['msg', 'keywords']].head().iterrows():
    print("--------------")
    print(f"{row['msg']}\n{row['keywords']}")

--------------
здравствуйте здравствуйте я застрахованное лицо я хотела бы узнать договор полиса и дата оплаты страховой премии мне по поводу кража у меня украли  #ЧИСЛО  тысяч мошенники я как бы чтоб страховой написать заявление то есть подать на страховой случай я правильно понимаю да да да мне должны указать договор полис я по телефону это все как то оплачивала договор вам оплатить нужно правильно понимаю нет у меня мошенники сняли  #ЧИСЛО  тысяч и как бы я хочу в страховую службу написать заявление мне нужно указать договор полис и дата оплаты страховой премии когда оплачивалась то есть вам нужен номер договора да номер договора мы не сообщаем как же мне заявление для по поводу кражи вы сказали вам звонить в сбербанк щас сижу так одна минута деньги  #ЧИСЛО  когда позвонит еще послан от обратиться этот номер договора номер договора говорит не сообщаем а потом и одну минуточку подождите не сообщаем а потом что мне делать и для документации не ладно сейчас минуточку подождите сейчас в